In [1]:
#dl.ipynb
#
#by Joe Hahn
#jmh.datasciences@gmail.com
#24 January 2018

In [18]:
#import matplotlib pandas etc
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import numpy as np
import pandas as pd
from matplotlib import rcParams
sns.set(font_scale=1.5, font='DejaVu Sans')
pd.set_option('display.max_columns', None)
pd.set_option('display.height', None)
pd.set_option('display.width', None)

height has been deprecated.



In [8]:
#read books dataframe
import pickle
with open('sentences.pkl', 'rb') as fp:
    sentences = pickle.load(fp)
print 'number of sentences = ', len(sentences)
sentences.head()

number of sentences =  1291287


,author,input_file,sentence,title
0,Shakespeare,iso/etext00/00ws110.txt,"Open the Gates vnto the Lord Protector, Or wee...",aShakespeare's First Folio/35 Plays
1,Shakespeare,iso/etext00/00ws110.txt,"For Humfrey; being dead, as he shall be, And H...",aShakespeare's First Folio/35 Plays
2,Shakespeare,iso/etext00/00ws110.txt,"Peace be with vs, least we be heauier Glou.",aShakespeare's First Folio/35 Plays
3,Shakespeare,iso/etext00/00ws110.txt,Speake thou no more if all the rest will speed...,aShakespeare's First Folio/35 Plays
4,Shakespeare,iso/etext00/00ws110.txt,"Further I will not flatter you, my Lord, That ...",aShakespeare's First Folio/35 Plays


In [10]:
#count number of sentences in each book, preserving longest book for multi-book authors,
#then dropping all books shorter than 1000 sentences
sort_cols = ['author', 'title', 'input_file']
sentence_count = sentences.groupby(sort_cols).count()
sentence_count = pd.DataFrame(sentence_count.to_records()).sort_values(['author', 'sentence'], ascending=False)\
    .drop_duplicates(['author'])
print 'number of books in filtered booklist = ', len(sentence_count)
idx = sentence_count['sentence'] > 1000
sentence_count = sentence_count[idx]
print 'number of books in filtered booklist = ', len(sentence_count)
sentence_count.head(10)

number of books in filtered booklist =  335
number of books in filtered booklist =  296


,author,title,input_file,sentence
566,various authors,Original Narratives of Early American History,iso/etext00/mohwk10.txt,1512
564,the Grimm Brothers,Fairy Tales,iso/etext01/grimm10.txt,3000
563,the Bronte Sisters,eBook of Jane Eyre,iso/etext98/janey11.txt,3000
562,such missionaries must lead to paradise!,"The Confessions of Rousseau, v12",iso/etext03/jj13b10.txt,3000
561,Yei Theodora Ozaki,Japanese Fairy Tales,iso/etext03/jpnft10.txt,2661
560,Xenophon,Cyropaedia,iso/etext00/cyrus10.txt,3000
558,Wollstonecraft,"Letters on Sweden, etc.",iso/etext02/ltswd10.txt,1462
557,William Stewart Rose,Orlando Furios,iso/etext96/orfur10.txt,3000
556,William S. Gilbert,: Complete Plays of Gilbert and Sullivan #2,iso/etext97/cpogs10.txt,3000
555,William Makepeace Thackeray,Vanity Fair,iso/etext96/vfair12.txt,3000


In [11]:
#this join preserves only one book (the longest) by each author exceeding 1000 sentences
sentences_sub = sentences.merge(sentence_count[sort_cols], on=sort_cols, how='inner')\
    .sort_values(sort_cols)[sort_cols + ['sentence']]
print 'number of sentences = ', len(sentences_sub)
sentences_sub.head(5)

number of sentences =  788002


,author,title,input_file,sentence
327411,A Religious of the Ursuline Community,The Life of the Venerable Mother Mary of the I...,iso/etext04/8vnmm10.txt,CHAPTER I.
327412,A Religious of the Ursuline Community,The Life of the Venerable Mother Mary of the I...,iso/etext04/8vnmm10.txt,"Among the many claimants for her hand, one had..."
327413,A Religious of the Ursuline Community,The Life of the Venerable Mother Mary of the I...,iso/etext04/8vnmm10.txt,Five months only before the Mother of the Inca...
327414,A Religious of the Ursuline Community,The Life of the Venerable Mother Mary of the I...,iso/etext04/8vnmm10.txt,"At length, sixteen of the colonists headed by ..."
327415,A Religious of the Ursuline Community,The Life of the Venerable Mother Mary of the I...,iso/etext04/8vnmm10.txt,"she exclaimed, ""surely favours like these are ..."


In [13]:
#check
sentence_count = sentences_sub.groupby(sort_cols).count()
cols = ['author', 'title', 'sentence']
sentence_count = pd.DataFrame(sentence_count.to_records()).sort_values(['author', 'title'])[cols]
print 'number of books by 1 author = ', len(sentence_count)
sentence_count.head()

number of books by 1 author =  296


,author,title,sentence
0,A Religious of the Ursuline Community,The Life of the Venerable Mother Mary of the I...,2713
1,A.H. Clough,PlutarchLives,3000
2,Abraham Lincoln,"The Writings of Lincoln, v7",3000
3,Adam Smith,Wealth of Nations,3000
4,Adolphus William Ward,Chaucer,1704


In [23]:
#randomize sentence order
sentences_ran = sentences_sub.sample(frac=1)
sentences_ran.head()

,author,title,input_file,sentence
421998,Thucydides,The History of the Peloponnesian War,iso/etext04/plpwr10.txt,Between ourselves and the Athenians alliance b...
778738,Chesterton,The Man Who Knew Too Much,iso/etext99/mwktm10a.txt,Neither of them thought it necessary to mentio...
535034,Arthur Guy Empey,Over The Top,iso/etext05/ovtop10.txt,went an improperly cut wire.
152360,Frederic Austin Ogg,"The Old Northwest, A Chronicle of the Ohio Val...",iso/etext02/oldno10.txt,From the Lakes to the Gulf redskins rose for v...
415669,Firebaugh,"Satyricon of Petronius Arbiter, Complete #8",iso/etext04/pas8w10.txt,"vi, 121 et seq."


In [42]:
#train-test-validate split
train_size = 0.5
rn_seed = 23
from sklearn.model_selection import train_test_split
train, test = train_test_split(sentences_ran, train_size=train_size, random_state=rn_seed)
validate, test = train_test_split(test, test_size=0.67, random_state=rn_seed)
print sentences_ran.shape
print train.shape
print test.shape
print validate.shape

(788002, 4)
(394001, 4)
(263981, 4)
(130020, 4)


In [45]:
#train word2vec model
train_sentences = train['sentence'].tolist()
print train_sentences[0:5]
from gensim.models import Word2Vec
w2v_model = Word2Vec(train_sentences, size=100, alpha=0.025, min_count=1)

['"The Manor, sir "--"The Manor!', '162.', 'We have no evidence beyond his own statement that he was a practical observer.', 'Why is the cost of government increasing  2.', 'I therefore determined to go to Besancon, and take some lessons from the Abbe Blanchard, and the idea appeared so rational to me, that I soon made Madam de Warrens of the same opinion, who immediately set about the preparations for my journey, in the same style of profusion with which all her plans were executed.']


In [49]:
w2v_model